# Ch12. 허프 변환


## 허프라인 검출(직선)  
- lines =cv.Houghlines(image, rho, theta, threshold)  

  - image : 8 비트 바이너리 이미지  
  - rho : r의 범위 0에서 1사이의 실수 입력  
  - theta : 0에서 180 사이의 실수를 입력, 라디안 단위임  
  - threshold : 극좌표계에서 곡선이 만나는 기준, 값이 클수록 정확도는 올라가지만 검출되는 직선의 개수가 줄어듦

In [1]:
import math
import cv2 as cv
import numpy as np

    

img_src = cv.imread("./12.1/test.jpg", cv.IMREAD_GRAYSCALE)
# Canny 함수를 통해서 엣지를 검출
img_edge = cv.Canny(img_src, 50, 150)
# 검출한 edge를 Grayscale에서 BGR로 변환
# 허프라인 검출하면, r, theta,
img_result = cv.cvtColor(img_edge, cv.COLOR_GRAY2BGR)
img_result_P = np.copy(img_result)
# 허프라인은 캐니 엣지 검출한 결과를 입력으로 받는다(중요)
lines = cv.HoughLines(img_edge, 1, np.pi / 180, 150)
# 1: 거리 해상도의 역수, 이게 픽셀단위, 1은 거리측정시 1픽셀 단위로 측정함
# 1은 거리측정 1픽셀 단위로 측정 하겠다.
# np.py/180 : 각도해상도, 각도차원의 분해능,
# np.pi/180 1도 간격
# 150 : 임계값 : 직선으로 간주되기 위해 지지하는 투표의 최소갯수(임계값이 높을수록 좀더 분명한 직선 검출)

# 검출된 직선을 표시해주는 부분
if lines is not None:
    for i in range(0, len(lines)):
        rho = lines[i][0][0]
        theta = lines[i][0][1]
        a = math.cos(theta)
        b = math.sin(theta)
        x0 = a * rho
        y0 = b * rho

        pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
        pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))
        
        cv.line(img_result, pt1, pt2, (0,0,255), 3)


cv.imshow("Source", img_src)
cv.imshow("Standard Hough Line Transform", img_result)

cv.waitKey()


-1

## 허프라인(HoubhLineP)  

- 모든 점이 아니라 임의의 점을 사용하여 직선을 검출
- lines = cv.HoughLinesP(image, rho, theta, threshold,lines, minLineLength, maxLineGap)
  - image : 이미지  
  - rho : 위 허프라인과 같음  
  - theta : 위 허프라인과 같음  
  - threshold : 극좌표계에서 곡선이 만나는 기준, 값이 클수록 정확도는 높지만 검출되는 갯수는 감소  
  - minLineLength : 최소선의 길이, 이것 이상의 직선을 검출  
  - MaxLineGap : 찾은 직선이 이값 이상 떨어져 있어야 별개의 선으로 간주함 

In [2]:
import math
import cv2 as cv
import numpy as np

    

img_src = cv.imread("./12.1/test.jpg", cv.IMREAD_GRAYSCALE)

img_edge = cv.Canny(img_src, 50, 150)

img_result = cv.cvtColor(img_edge, cv.COLOR_GRAY2BGR)



linesP = cv.HoughLinesP(img_edge, 1, np.pi / 180, 50, None, 50, 5)

if linesP is not None:
    for i in range(0, len(linesP)):
        # linesP가 출력인데, I행 0번째 열이 점의 좌표값
        l = linesP[i][0]
        # 끝점 1 l[0], l[1]
        # 끝점 2 l[2], l[3] 총 2개의 점으로 직선을 그리는 부분
        cv.line(img_result, (l[0], l[1]), (l[2], l[3]), (0,0,255), 3, cv.LINE_AA)

cv.imshow("Source", img_src)
cv.imshow("Probabilistic Line Transform", img_result)

cv.waitKey()


-1

In [3]:
import numpy as np
import cv2 as cv

img_gray = cv.imread('./12.2/test.jpg', cv.IMREAD_GRAYSCALE)
img_gray = cv.medianBlur(img_gray,5)
img_color = cv.cvtColor(img_gray,cv.COLOR_GRAY2BGR)

circles = cv.HoughCircles(img_gray,cv.HOUGH_GRADIENT,1,20,
                            param1=50,param2=35,minRadius=0,maxRadius=0)
circles = np.uint16(np.around(circles))

for c in circles[0,:]:

    center = (c[0],c[1])
    radius = c[2]
    
    # 바깥원
    cv.circle(img_color,center,radius,(0,255,0),2)
    
    # 중심원
    cv.circle(img_color,center,2,(0,0,255),3)

cv.imshow('detected circles',img_color)
cv.waitKey(0)
cv.destroyAllWindows()